In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = """You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. 
                    Always be accurate. If you don't know the answer, say so."""

ticket_prices_dict = {"texas":"$799", "ohio":"$599", "mumbai":"$499", "london":"$899", "tokyo":"$799"}

ticket_availability_dict = {"texas": "30", "ohio":"20", "mumbai":"15", "london":"10", "tokyo":"0"}

In [4]:
def ticket_price(city):
    print(f"Tool ticket_price called for {city}")
    return ticket_prices_dict.get(city.lower(),"Unknown")

In [5]:
def get_ticket_availability(city):
    print(f"Tool get_ticket_availability called for {city}")
    available = city.lower()
    return ticket_availability_dict.get(available, "Unknown")

In [6]:
def ticket_booking(city, number_of_available_tickets, number_of_tickets):
    print(f"Tool ticket_booking called for {city} and number_of_available_tickets are {number_of_available_tickets} and number_of_tickets are {number_of_tickets}")
    if int(number_of_available_tickets if number_of_available_tickets!="Unknown" else "0")-int(number_of_tickets) < 0:
        return "Currently tickets are not available for "+city+"."
    else:
        ticket_availability_dict[city.lower()] = str(int(number_of_available_tickets)-int(number_of_tickets))
        return "Your booking has been confirmed for "+city+"."
        

In [7]:
ticket_price_function = {
    "name": "ticket_price",
    "description": "Get the price of a return ticket to the mentioned city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

ticket_availability_function = {
    "name": "get_ticket_availability",
    "description": "Get number of tickets that are available for booking to the mentioned city. Call this whenever you need to know available tickets for booking, for example when a customer asks 'How many tickets are available for booking for city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to book tickets for",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

ticket_booking_function = {
    "name": "ticket_booking",
    "description": "Book the tickets to the mentioned city. Call this whenever you need to book tickets, for example when a customer asks 'I want to book tickets for this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            },
            "number_of_available_tickets": {
                "type": "string",
                "description": "Total number of tickets that are available for bookong"
            },
            "number_of_tickets": {
                "type": "string",
                "description": "The number of tickets that the customer wants to book"
            }
        },
        "required": ["city","number_of_available_tickets", "number_of_tickets"],
        "additionalProperties": False
    }
}

In [8]:
tools = [{"type": "function", "function": ticket_price_function}, 
         {"type": "function", "function": ticket_availability_function},
         {"type": "function", "function": ticket_booking_function}]

In [9]:
def handle_tools(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    function_name = json.dumps(tool_call.function.name)
    city = arguments.get('city')
    num_of_tickets = arguments.get('number_of_tickets')  

    if function_name.replace('"','') == "ticket_price":
        price = ticket_price(city)
        response = { 
            "role": "tool",
            "content": json.dumps({"city": city,"ticket_price": price}),
            "tool_call_id": tool_call.id
        }
    elif function_name.replace('"','') == "ticket_booking":
        number_of_available_tickets = get_ticket_availability(city)
        result = ticket_booking(city, number_of_available_tickets, num_of_tickets)
        response = {
            "role": "tool",
            "content": json.dumps({"city": city,"booking": result}),
            "tool_call_id": tool_call.id
        }

    else:
        available_tickets = get_ticket_availability(city)
        response = {
            "role": "tool",
            "content": json.dumps({"city": city,"number_of_available_tickets": available_tickets}),
            "tool_call_id": tool_call.id
        }
    return response, city

In [10]:
def chat(message, history):
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        function_name = json.dumps(tool_call.function.name)
        print(function_name)
        city = arguments.get('city')
        number_of_tickets = arguments.get('number_of_tickets')

        if function_name.replace('"','') == "book_ticket":
            response, city = handle_tools(message)
            messages.append(message)
            messages.append(response)
            # image = artist(city)
            response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

        elif function_name.replace('"','') == "ticket_price_function":
            price = ticket_price(city)
            availability = get_ticket_availability(city)
            response = {
                        "role": "tool",
                        "content": json.dumps({"destination_city": city,"price": price,"availability": availability}),
                        "tool_call_id": tool_call.id
                        }
            messages.append(message)
            messages.append(response)
            print(messages)
            response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)                    
        else:    
            response, city = handle_tools(message)
            messages.append(message)
            messages.append(response)
            # image = artist(city)
            response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## after booking the ticket the ticket_availability_dict got updated

In [12]:
ticket_availability_dict

{'texas': '30', 'ohio': '20', 'mumbai': '15', 'london': '5', 'tokyo': '0'}